In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from collections import Counter

In [4]:
y_test = pd.read_csv("../input/titanic/gender_submission.csv",index_col="PassengerId")
x_train=pd.read_csv("../input/titanic/train.csv")
x_test=pd.read_csv("../input/titanic/test.csv")

In [5]:
x_train.drop([ 'Name', 'Ticket', 'Cabin'], 'columns', inplace=True)

x_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [6]:
x_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [7]:
# Filling missing value of Age 

## Fill Age with the median age of similar rows according to Pclass, Parch and SibSp
# Index of NaN age rows
index_NaN_age = list(x_train["Age"][x_train["Age"].isnull()].index)

for i in index_NaN_age :
    age_med = x_train["Age"].median()
    age_pred =x_train["Age"][((x_train['SibSp'] == x_train.iloc[i]["SibSp"]) & (x_train['Parch'] == x_train.iloc[i]["Parch"]) & (x_train['Pclass'] == x_train.iloc[i]["Pclass"]))].median()
    if not np.isnan(age_pred) :
        x_train['Age'].iloc[i] = age_pred
    else :
        x_train['Age'].iloc[i] = age_med

In [8]:
x_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
dtype: int64

In [9]:
x_train["Embarked"] = x_train["Embarked"].fillna("S")

In [10]:
#outlier detection
#tukey method

def detect_outliers(df, n, features):
    outlier_indices = []
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(x_train,2,["Age","SibSp","Parch","Fare"])

In [11]:
x_train.loc[Outliers_to_drop]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
268,269,1,1,female,58.00,0,1,153.4625,S
305,306,1,1,male,0.92,1,2,151.5500,S
438,439,0,1,male,64.00,1,4,263.0000,S
587,588,1,1,male,60.00,1,1,79.2000,C
659,660,0,1,male,58.00,0,2,113.2750,C
745,746,0,1,male,70.00,1,1,71.0000,S
27,28,0,1,male,19.00,3,2,263.0000,S
88,89,1,1,female,23.00,3,2,263.0000,S
159,160,0,3,male,28.00,8,2,69.5500,S
180,181,0,3,female,28.00,8,2,69.5500,S


In [12]:
# Drop outliers
x_train = x_train.drop(Outliers_to_drop, axis = 0).reset_index()

In [13]:
x_train.isnull().sum()

index          0
PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [14]:
x_train = pd.get_dummies(x_train, columns=['Embarked'])

x_train.head()

,index,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,1,0,3,male,22.0,1,0,7.2500,0,0,1
1,1,2,1,1,female,38.0,1,0,71.2833,1,0,0
2,2,3,1,3,female,26.0,0,0,7.9250,0,0,1
3,3,4,1,1,female,35.0,1,0,53.1000,0,0,1
4,4,5,0,3,male,35.0,0,0,8.0500,0,0,1


In [15]:
x_train.index = x_train['PassengerId']

In [16]:
x_train.drop('PassengerId',axis = 1,inplace=True)

In [17]:
from sklearn import preprocessing

label_encoding = preprocessing.LabelEncoder()
x_train['Sex'] = label_encoding.fit_transform(x_train['Sex'].astype(str))

x_train.head()

,index,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,
1,0,0,3,1,22.0,1,0,7.2500,0,0,1
2,1,1,1,0,38.0,1,0,71.2833,1,0,0
3,2,1,3,0,26.0,0,0,7.9250,0,0,1
4,3,1,1,0,35.0,1,0,53.1000,0,0,1
5,4,0,3,1,35.0,0,0,8.0500,0,0,1


In [18]:
x_train.drop('index',axis = 1,inplace=True)

In [19]:
y_train = x_train['Survived']

In [20]:
y_train

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
887    0
888    1
889    0
890    1
891    0
Name: Survived, Length: 875, dtype: int64

In [21]:
x_train.drop('Survived',axis = 1,inplace=True)

In [22]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,3,1,22.0,1,0,7.2500,0,0,1
2,1,0,38.0,1,0,71.2833,1,0,0
3,3,0,26.0,0,0,7.9250,0,0,1
4,1,0,35.0,1,0,53.1000,0,0,1
5,3,1,35.0,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...
887,2,1,27.0,0,0,13.0000,0,0,1
888,1,0,19.0,0,0,30.0000,0,0,1
889,3,0,13.5,1,2,23.4500,0,0,1


In [23]:
x_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [24]:
x_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [25]:
x_test.drop([ 'Name', 'Ticket', 'Cabin'], 'columns', inplace=True)

x_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [26]:
# Filling missing value of Age 

## Fill Age with the median age of similar rows according to Pclass, Parch and SibSp
# Index of NaN age rows
index_NaN_age = list(x_test["Age"][x_test["Age"].isnull()].index)

for i in index_NaN_age :
    age_med = x_test["Age"].median()
    age_pred =x_test["Age"][((x_test['SibSp'] == x_test.iloc[i]["SibSp"]) & (x_test['Parch'] == x_test.iloc[i]["Parch"]) & (x_test['Pclass'] == x_test.iloc[i]["Pclass"]))].median()
    if not np.isnan(age_pred) :
        x_test['Age'].iloc[i] = age_pred
    else :
        x_test['Age'].iloc[i] = age_med

In [27]:
x_test["Fare"] = x_test["Fare"].fillna(x_test["Fare"].median())

In [28]:
x_test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [29]:
x_test.index = x_test['PassengerId']

In [30]:
x_test = x_test.drop(['PassengerId'],axis=1)

In [31]:
x_test = pd.get_dummies(x_test , columns=['Embarked'])

x_test .head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,3,male,34.5,0,0,7.8292,0,1,0
893,3,female,47.0,1,0,7.0000,0,0,1
894,2,male,62.0,0,0,9.6875,0,1,0
895,3,male,27.0,0,0,8.6625,0,0,1
896,3,female,22.0,1,1,12.2875,0,0,1


In [32]:
from sklearn import preprocessing

label_encoding = preprocessing.LabelEncoder()
x_test['Sex'] = label_encoding.fit_transform(x_test['Sex'].astype(str))
x_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,3,1,34.5,0,0,7.8292,0,1,0
893,3,0,47.0,1,0,7.0000,0,0,1
894,2,1,62.0,0,0,9.6875,0,1,0
895,3,1,27.0,0,0,8.6625,0,0,1
896,3,0,22.0,1,1,12.2875,0,0,1


In [33]:
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LogisticRegression 

In [34]:
x_train, x_hold_out = np.split(x_train, [int(.7*len(x_train)) ])

In [35]:
y_train, y_hold_out = np.split(y_train, [int(.7*len(y_train)) ])

In [36]:
x_hold_out

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
624,3,1,21.0,0,0,7.8542,0,0,1
625,3,1,21.0,0,0,16.1000,0,0,1
626,1,1,61.0,0,0,32.3208,0,0,1
627,2,1,57.0,0,0,12.3500,0,1,0
628,1,0,21.0,0,0,77.9583,0,0,1
...,...,...,...,...,...,...,...,...,...
887,2,1,27.0,0,0,13.0000,0,0,1
888,1,0,19.0,0,0,30.0000,0,0,1
889,3,0,13.5,1,2,23.4500,0,0,1


In [37]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,3,1,22.0,1,0,7.2500,0,0,1
2,1,0,38.0,1,0,71.2833,1,0,0
3,3,0,26.0,0,0,7.9250,0,0,1
4,1,0,35.0,1,0,53.1000,0,0,1
5,3,1,35.0,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...
619,2,0,4.0,2,1,39.0000,0,0,1
620,2,1,26.0,0,0,10.5000,0,0,1
621,3,1,27.0,1,0,14.4542,1,0,0


In [38]:
y_train

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
619    1
620    0
621    0
622    1
623    1
Name: Survived, Length: 612, dtype: int64

In [39]:
x_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,3,1,34.5,0,0,7.8292,0,1,0
893,3,0,47.0,1,0,7.0000,0,0,1
894,2,1,62.0,0,0,9.6875,0,1,0
895,3,1,27.0,0,0,8.6625,0,0,1
896,3,0,22.0,1,1,12.2875,0,0,1
...,...,...,...,...,...,...,...,...,...
1305,3,1,24.0,0,0,8.0500,0,0,1
1306,1,0,39.0,0,0,108.9000,1,0,0
1307,3,1,38.5,0,0,7.2500,0,0,1


In [40]:
y_test

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [41]:
clf1 = KNeighborsClassifier(n_neighbors=10)
clf2 = RandomForestClassifier(n_estimators=50)
clf3 = GaussianNB()

for clf in (clf1, clf2, clf3):
    clf.fit(x_train, y_train)

In [42]:
def get_predictions(x, y):
    pred_result = pd.DataFrame()
    
    i = 1
    for clf in (clf1, clf2, clf3):

        y_pred = clf.predict(x)
        
        print(clf.__class__.__name__, accuracy_score(y, y_pred))
        
        pred_result.insert(i - 1, 'y_pred_' + str(i), y_pred)
        
        i += 1
        
    return pred_result

In [43]:
pred_result = get_predictions(x_hold_out, y_hold_out)

KNeighborsClassifier 0.714828897338403
RandomForestClassifier 0.8288973384030418
GaussianNB 0.8136882129277566


In [44]:
pred_result.head()

,y_pred_1,y_pred_2,y_pred_3
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,1,1


In [45]:
x_stack_train = pred_result

y_stack_train = y_hold_out

In [46]:
clf_stack = LogisticRegression(solver='lbfgs', C=1, max_iter=200)

clf_stack.fit(x_stack_train, y_stack_train)

LogisticRegression(C=1, max_iter=200)

In [47]:
pred_result_test = get_predictions(x_test, y_test)

KNeighborsClassifier 0.638755980861244
RandomForestClassifier 0.868421052631579
GaussianNB 0.9019138755980861


In [48]:
pred_result_test

,y_pred_1,y_pred_2,y_pred_3
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,1,1
...,...,...,...
413,0,0,0
414,1,1,1
415,0,0,0
416,0,0,0


In [49]:
x_stack_test = pred_result_test

y_stack_pred = clf_stack.predict(x_stack_test)

In [50]:
y_stack_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [51]:
y_stack_pred = pd.DataFrame(y_stack_pred)

In [52]:
y_stack_pred.index = x_test.index

In [53]:
y_stack_pred

,0
PassengerId,
892,0
893,0
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [54]:
y_stack_pred.columns = ["Survived"]

In [55]:
y_stack_pred

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [56]:
y_stack_pred.to_csv("ensemble_python_stacking.csv")